In [1]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()
# Deep-Q learning Agent
class DQNAgent:
    def __init__(self):
        self.memory = []
        self.gamma = 0.9  # decay rate
        self.epsilon = 1  # exploration
        self.epsilon_decay = .995
        self.epsilon_min = 0.1
        self.learning_rate = 0.0001
        self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Flatten(input_shape=(1,300,300)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(5, activation='linear'))
        model.compile(loss='mse',
                      optimizer=RMSprop(lr=self.learning_rate))
        print("Now we build the model")
        # model = Sequential()
        # model.add(Convolution2D(32, 8, 8, subsample=(4,4),input_shape=(1,300,300), border_mode='same'))
        # model.add(Activation('relu'))
        # model.add(Convolution2D(64, 4, 4, subsample=(2,2), border_mode='same'))
        # model.add(Activation('relu'))
        # model.add(Convolution2D(64, 3, 3, subsample=(1,1), border_mode='same'))
        # model.add(Activation('relu'))
        # model.add(Flatten())
        # model.add(Dense(512))
        # model.add(Activation('linear'))
        # model.add(Dense(5))

        # adam = Adam(lr=1e-6)
        # model.compile(loss='mse',optimizer=adam)
        print("We finish building the model")
        self.model = model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return int(random.uniform(0, 3))
        act_values = self.model.predict(state)
        return int(np.argmax(act_values[0]))  # returns action
    
    def replay(self, batch_size):
        batches = min(batch_size, len(self.memory))
        batches = np.random.choice(len(self.memory), batches)
        for i in batches:
            state, action, reward, next_state, done = self.memory[i]
            target = reward
            if not done:
                target = reward + self.gamma * np.argmax(self.model.predict(next_state))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            # print ("started training episode")
            self.model.fit(state, target_f, nb_epoch=1, verbose=0)
            # print ("finished training episode")
        self.model.save_weights("modelSmove.h5")
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
from PIL import Image
import pyscreenshot as ImageGrab
import numpy as np
import pytesseract
import os
import random
import time
import keras
import win32api, win32con
from random import sample
from keras import backend as K
import random
import keras
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Activation
from keras.optimizers import sgd, Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import sys
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
K.set_image_dim_ordering('th')

orig_stdout = sys.stdout
f = open('out.txt', 'w')
sys.stdout = f


x_pad = 1500
y_pad = 50

def preprocess(im1):
    im1= im1.resize((300,300))
    im1 = im1.convert('L')
    print("[INFO] loading and preprocessing image...")
    image = img_to_array(im1)
    image = image.reshape((1,) + image.shape)  # this is a Numpy array with shape (1, 3, 300, 300)
    test_ob = ImageDataGenerator(rescale=1./255)
    X=[]
    for batch in test_ob.flow(image, batch_size=1):
        X= batch
        break
    return X
def checkGameOver():
    im = screenGrab()
    left = 0
    top = 130
    width = 230
    height = 50
    box = (left, top, left+width, top+height)
    im2 = im.crop(box)
    text = pytesseract.image_to_string(im2)
    GAME_OVER = ['S','c','o','r','e',':']     
    if any(val in text for val in GAME_OVER):
        # print "=====Check game over=======\n"
        # print('Game over encountered')
        # print "===========================\n"
        return True
    else:
        # print "=====Check game over=======\n"
        # print('Countinue playing the game')
        # print "===========================\n"
        return False

def getScore():
    im = screenGrab()
    left = 0
    top = 25
    width = 40
    height = 40
    box = (left, top, left+width, top+height)
    im2 = im.crop(box)
    text = pytesseract.image_to_string(im2)
    # print "=====Get score=============\n"
    # print('the score is :' + text )
    # print "===========================\n"
    try:
        return int(text)
    except ValueError:
        # print 'errror encountered in reading score'
        return -1

def closeAd():
    """
        still have to write code for closing the add
    """
    x,y = -803, -62
    mousePos((x, y))
    time.sleep(.1)
    leftClick()
    time.sleep(.1)
    # print "===========================\n"
    # print('add closed ')
    # print "===========================\n"

def restartGame():
    """
        click restart game 
    """
    x,y = -87, 565
    mousePos((x, y))
    time.sleep(.1)
    leftClick()
    # print "===========================\n"
    # print('Game restarted ')
    # print "===========================\n"

def pauseGame():
    """
        click pause/play game 
    """
    x,y = 7, 345
    mousePos((x, y))
    time.sleep(.1)
    leftClick()
    # print "===========================\n"
    # print('Game restarted ')
    # print "===========================\n"
    

def screenGrab():
    x_pad = 1300
    y_pad = 380
    x=1536
    y=690
    box = (x_pad+1, y_pad+1, x, y)
    im = ImageGrab.grab(box)
    return im

def leftClick():
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.1)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    # print "left Click."

def rightClick():
    win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTDOWN,0,0)
    time.sleep(.1)
    win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTUP,0,0)
    # print "left Click."
    
def leftDown():
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.1)
    # print 'left Down'
         
def leftUp():
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    time.sleep(.1)
    # print 'left release'

# def mousePos(cord):
#     win32api.SetCursorPos((x_pad + cord[0], y_pad + cord[1])

def swipeDown():
    x,y= -82, 554
    x1,y1 = -82,600
    mousePos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.1)
    mousePos((x1,y1))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    # print 'swipe down complete'

def swipeUp():
    x1,y1= -82, 554
    x,y = -82,600
    mousePos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.1)
    mousePos((x1,y1))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    # print 'swipe up complete'

def swipeRight():
    x,y= -82, 554
    x1,y1= 0,554
    mousePos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.1)
    mousePos((x1,y1))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    # print 'swipe right complete'

def swipeLeft():
    x1,y1= -82, 554
    x,y= 0,554
    mousePos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,0,0)
    time.sleep(.1)
    mousePos((x1,y1))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,0,0)
    # print 'swipe left complete'

def mousePos(cord):
    win32api.SetCursorPos((x_pad + cord[0], y_pad + cord[1]))

def get_cords():
    x,y = win32api.GetCursorPos()
    print (x,y)
    x = x - x_pad
    y = y - y_pad
    print (x,y)

def take_action(action):
    """ executes action action. """
    if action == 0:
        swipeUp()
    elif action == 1:
        swipeDown()            
    elif action == 2:
        swipeLeft()
    elif action == 3:
        swipeRight()
    elif action == 4:
        pass
    else:
        print('Unrecognized action %d' % action)

def reset():
    """ Repeats NO-OP action until a new episode begins. """
    restartGame()
    time.sleep(.5)
    state = screenGrab()
    pauseGame()# pause play
    return state

def step(action, score):
    status = checkGameOver()
    # print status
    s = getScore()
    if status:
        reward=0
        s= score
    else:
        if s>score:
            reward = 1000
        else:
            reward=-250
        pauseGame()# restart play
        take_action(action)
        time.sleep(0.2)
        
        
        s = getScore()  
        if s==-1:
            s=score
        if status:
            reward=0
            s= score
        else:
            if s>score:
                reward = 1000
            else:
                reward=-250
    state = screenGrab()
    episode_over = status
    time.sleep(0.2)
    pauseGame() # pause play
    return state, reward, episode_over,s, {}    

    
if __name__ == '__main__':
    # initialize environment and the agent
    agent = DQNAgent()
    episodes = 1200

    # Iterate the game
    for e in range(episodes):

        # reset state in the beginning of each game
        state = reset()
        state = preprocess(state)
        score = 0
        # time_t represents each frame of the game
        # Our goal is to keep the pole upright as long as possible
        # the more time_t the more score
        for time_t in range(5000):
            # turn this on if you want to render
            # env.roender()

            # Decide action
            action = agent.act(state)
            
            # Advance the game to the next frame based on the action.
            next_state, reward, done,score, _ = step(action, score)
            next_state = preprocess(next_state)

            # (reward defaults to 1)
            # reward the agent 1 for every frame it lived
            # and punish -100 for dying
            reward = -5000 if done else reward

            # Remember the previous state, action, reward, and done
            agent.remember(state, action, reward, next_state, done)

            # make next_state the new current state for the next frame.
            state = copy.deepcopy(next_state)

            # done becomes True when the game ends
            # ex) The agent drops the pole
            if done:
                # print the score and break out of the loop
                print("episode: {}/{}, score: {}"
                      .format(e, episodes, score))
                break
        # train the agent with the experience of the episode
        agent.replay(32)

Using TensorFlow backend.
C:\Users\suhag\AppData\Local\conda\conda\envs\py35\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '
